**About** : This notebook is used to do inference on test data

OUTDATED

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import gc
import os
import ast
import sys
import cv2
import glob
import json
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
warnings.simplefilter("ignore", UserWarning)

In [ ]:
from params import *
from utils.plots import *

from utils.logger import Config
from inference.test import inference_test

## Exp

In [ ]:
EXP_FOLDERS = [
    LOG_PATH + "2021-11-10/21/",  # 0.3084 / 0.3046
]

THRESHOLDS_CONF = [0.3, 0.35, 0.7]  
THRESHOLDS_MASK = [0.45, 0.4, 0.45]

USE_TTA = True

## Data

In [ ]:
from data.preparation import prepare_extra_data
df = prepare_extra_data()

# df = pd.read_csv(DATA_PATH + "sample_submission.csv")
# df['img_path'] = DATA_PATH + "test/" + df['id'] + ".png"

In [ ]:
configs, weights = [], []

for exp_folder in EXP_FOLDERS:
    config = Config(json.load(open(exp_folder + "config.json", 'r')))
    config.model_config = exp_folder + config.model_config.split('/')[-1]
    config.data_config = exp_folder + config.data_config.split('/')[-1]
    configs.append(config)

    weights.append(sorted(glob.glob(exp_folder + "*.pt")))

## Main

In [ ]:
weights = [[weights[0][0]]]

In [ ]:
results = inference_test(df, configs, weights, use_tta=False)

## Viz

In [ ]:
from params import CELL_TYPES
from inference.post_process import post_process_preds
from utils.plots import plot_sample
from data.dataset import SartoriusInferenceDataset
from data.transforms import define_pipelines

In [ ]:
cells = []
for result in results:
    lens = [len(boxes_c) for boxes_c, masks_c in zip(result[0], result[1])]
    cell = np.argmax(lens)
    cells.append(cell)
    
sns.countplot(x=cells)
plt.xticks(range(3), CELL_TYPES)
plt.show()

In [ ]:
THRESHOLDS_CONF = [0.3, 0.35, 0.7]
THRESHOLDS_MASK = [0.45, 0.4, 0.45]

In [ ]:
pipelines = define_pipelines(config.data_config)
dataset = SartoriusInferenceDataset(df, transforms=pipelines['test_viz'], precompute_masks=False)
counts = []
for idx in range(len(dataset)):
    img = dataset[idx]['img'][0].numpy().transpose(1, 2, 0)
    img = (img - img.min()) / (img.max() - img.min())
    
    lens = [len(boxes_c) for boxes_c, masks_c in zip(results[idx][0], results[idx][1])]
    counts.append(lens)
    
    cell_type = np.argmax(lens)
    
#     if lens[1] < 5 and lens[2] < 5:
#         continue

    print(idx, lens)
    
    masks, boxes, _ = post_process_preds(results[idx], THRESHOLDS_CONF, THRESHOLDS_MASK, remove_overlap=False)

    plt.figure(figsize=(15, 15))
    plot_sample(img, masks.astype(int), boxes, plotly=False)
    plt.axis(False)
    plt.title(CELL_TYPES[cell_type])
    plt.show()
    
    if idx > 10:
        break

In [ ]:
counts = np.array(counts)
counts.shape
np.save("../output/counts_livecell_shsy5y.npy", counts)